In [1]:
import torch
from torch import nn


#### Depthwise model

In [2]:
class Chomp1d(nn.Module):
    """PyTorch does not offer native support for causal convolutions, so it is implemented (with some inefficiency) by simply using a standard convolution with zero padding on both sides, and chopping off the end of the sequence."""
    def __init__(self, chomp_size) -> None:
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class FirstBlock(nn.Module):
    def __init__(self, target, n_inputs, n_outputs, kernel_size, stride, dilation, padding, device = 'cpu'):
        super(FirstBlock, self).__init__()
        
        self.target = target
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation, groups=n_outputs, device=device)

        self.chomp1 = Chomp1d(padding)
        self.net = nn.Sequential(self.conv1, self.chomp1)      
        self.relu = nn.PReLU(n_inputs, device=device)
        self.device = device
        self.init_weights()

    def init_weights(self) -> None:
        """Initialize weights"""
        self.conv1.weight.data.normal_(0, 0.1, generator=torch.Generator(device=self.device))
        
    def forward(self, x) -> torch.Tensor:
        """Forward pass"""
        out = self.net(x)
        return self.relu(out)    

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, device='cpu') -> None:
        super(TemporalBlock, self).__init__()
       
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation, groups=n_outputs, device=device)
        self.chomp1 = Chomp1d(padding)
        self.net = nn.Sequential(self.conv1, self.chomp1)
        self.relu = nn.PReLU(n_inputs, device=device)
        self.device = device
        self.init_weights()

    def init_weights(self) -> None:
        """Initialize weights"""
        self.conv1.weight.data.normal_(0, 0.1, generator=torch.Generator(device=self.device)) 
        

    def forward(self, x) -> torch.Tensor:
        """Forward residual pass of the temporal block"""
        out = self.net(x)
        return self.relu(out+x) #residual connection

class LastBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, device ='cpu') -> None:
        super(LastBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation, groups=n_outputs, device=device)
        self.chomp1 = Chomp1d(padding)
        self.net = nn.Sequential(self.conv1, self.chomp1)
        self.linear = nn.Linear(n_inputs, n_inputs, device=device)
        self.device = device
        self.init_weights()

    def init_weights(self) -> None:
        """Initialize weights"""
        self.linear.weight.data.normal_(0, 0.01, generator=torch.Generator(device=self.device)) 
        
    def forward(self, x) -> torch.Tensor:
        out = self.net(x)
        return self.linear(out.transpose(1,2)+x.transpose(1,2)).transpose(1,2) #residual connection

class DepthwiseNet(nn.Module):
    def __init__(self, target, num_inputs, num_levels, kernel_size=2, dilation_c=2, device='cpu') -> None:
        super(DepthwiseNet, self).__init__()
        layers = []
        in_channels = num_inputs
        out_channels = num_inputs
        for l in range(num_levels):
            dilation_size = dilation_c ** l
            if l==0:
                layers += [FirstBlock(target, in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, device=device)]
            elif l==num_levels-1:
                layers+=[LastBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, device=device)]
            
            else:
                layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, device=device)]

        self.network = nn.Sequential(*layers)

    def forward(self, x) -> torch.Tensor:
        """Forward pass of the network"""
        return self.network(x)


### AD-DSTCN Model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import random
import pandas as pd
import numpy as np
import heapq
import copy

class ADDSTCN(nn.Module):
    def __init__(self, target, input_size, num_levels, kernel_size, cuda, dilation_c,
                 lasso_lambda=1e-6):
        super(ADDSTCN, self).__init__()
        
        self.device = torch.device("cuda" if cuda else "cpu")

        self.lasso_lambda = lasso_lambda
        
        self.target=target
        self.dwn = DepthwiseNet(self.target, input_size, num_levels, kernel_size=kernel_size, dilation_c=dilation_c)
        self.pointwise = nn.Conv1d(input_size, 1, 1, device=self.device)

        #self._attention = torch.ones(input_size,1)
        self.fs_attention_logits = nn.Parameter(torch.ones(input_size, 1, device=self.device))
        
        if cuda:
            self.dwn = self.dwn.cuda()
                  
    def init_weights(self) -> None:
        self.pointwise.weight.data.normal_(0, 0.1)       
        
    def forward(self, x):
        
        # new variable for LASSO attention scores interpretability
        attention_scores = torch.sigmoid(self.fs_attention_logits)
        
        #y1=self.dwn(x*F.softmax(self.fs_attention, dim=0))
        y1=self.dwn(x*attention_scores)
        y1 = self.pointwise(y1) 
        return y1.transpose(1,2)
    
    def attention_regularization(self, p=1):
        """L1 penalty on attention scores [0,1]"""
        attention_scores = torch.sigmoid(self.fs_attention_logits)
        return self.lasso_lambda * torch.norm(attention_scores, p=1)
    
    def get_sparsity_stats(self):
        scores = torch.sigmoid(self.fs_attention_logits).detach().squeeze()
        
        return {
            'mean_attention': scores.mean().item(),
            'near_zero': (scores < 0.1).sum().item(),  # "Spente"
            'active': (scores > 0.5).sum().item(),     # "Attive"
            'active_idx': (scores > 0.5).nonzero(as_tuple=True)[0].cpu().numpy(),
            'sparsity_ratio': (scores < 0.1).float().mean().item()
        }
        
    def get_attention_scores(self):
        """Returns attention scores as a numpy array."""
        scores = torch.sigmoid(self.fs_attention_logits).detach().cpu().numpy()
        return scores.squeeze()

### TCDF Algorithm

In [4]:
from pyexpat import model
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import random
import pandas as pd
import numpy as np
import heapq
import copy
import os
import sys

def preparedata(file, target) -> tuple[Variable, Variable]:
    """Reads data from csv file and transforms it to two PyTorch tensors: dataset x and target time series y that has to be predicted."""
    df_data = pd.read_csv(file)
    df_y = df_data.copy(deep=True)[[target]]
    df_x = df_data.copy(deep=True)
    df_yshift = df_y.copy(deep=True).shift(periods=1, axis=0)
    df_yshift[target]=df_yshift[target].fillna(0.)
    df_x[target] = df_yshift
    data_x = df_x.values.astype('float32').transpose()    
    data_y = df_y.values.astype('float32').transpose()
    data_x = torch.from_numpy(data_x)
    data_y = torch.from_numpy(data_y)

    x, y = Variable(data_x), Variable(data_y)
    return x, y


def train(epoch, traindata, traintarget, modelname: ADDSTCN, optimizer,log_interval,epochs):
    """Trains model by performing one epoch and returns attention scores and loss."""

    modelname.train()
    x, y = traindata[0:1], traintarget[0:1]
        
    optimizer.zero_grad()
    epochpercentage = (epoch/float(epochs))*100
    output = modelname(x)

    
    loss_mse = F.mse_loss(output, y)
    loss_lasso = modelname.attention_regularization()
    loss = loss_mse + loss_lasso
    loss.backward()
    optimizer.step()
    
    # update lasso lambda
    modelname.lasso_lambda = modelname.lasso_lambda * 0.9999#min(1e-9, modelname.lasso_lambda * 0.9999)
    
    attentionscores = modelname.get_attention_scores()

    if epoch % log_interval ==0 or epoch % epochs == 0 or epoch==1:
        print('Epoch: {:2d} [{:.0f}%] \tLoss: {:.4e} - \tLambda: {:.4e}'.format(epoch, epochpercentage, loss, modelname.lasso_lambda))
        print("Get sparsity stats: ", modelname.get_sparsity_stats())

    return attentionscores, loss

def findcauses(target, cuda, epochs, kernel_size, layers, 
               log_interval, lr, optimizername, seed, dilation_c, significance, file):
    """Discovers potential causes of one target time series, validates these potential causes with PIVM and discovers the corresponding time delays"""

    print("\n", "Analysis started for target: ", target)
    torch.manual_seed(seed)
    
    X_train, Y_train = preparedata(file, target)
    X_train = X_train.unsqueeze(0).contiguous()
    Y_train = Y_train.unsqueeze(2).contiguous()

    input_channels = X_train.size()[1]
       
    targetidx = pd.read_csv(file).columns.get_loc(target)
          
    model = ADDSTCN(targetidx, input_channels, layers, kernel_size=kernel_size, cuda=cuda, dilation_c=dilation_c)
    if cuda:
        model.cuda()
        X_train = X_train.cuda()
        Y_train = Y_train.cuda()

    optimizer = getattr(optim, optimizername)(model.parameters(), lr=lr)    
    
    scores, firstloss = train(1, X_train, Y_train, model, optimizer,log_interval,epochs)
    firstloss = firstloss.cpu().data.item()
    for ep in range(2, epochs+1):
        scores, realloss = train(ep, X_train, Y_train, model, optimizer,log_interval,epochs)
        #if ep == 10:
        #    firstloss = realloss.cpu().data.item()
    realloss = realloss.cpu().data.item()
    
    print("Scores: ", scores)
    print("First loss: ", firstloss, " - Real loss: ", realloss)
    print('-'*50)
    
    #s = sorted(scores.view(-1).cpu().detach().numpy(), reverse=True)
    #print("Scores: ", s)
    #indices = np.argsort(-1 * scores.view(-1).cpu().detach().numpy()) # -1 for descending order
    #print("Indices: ", indices)
    
    # get indices of scores that are greater than 0.5
    potentials = list(np.where(scores > 0.5)[0])
    
    #attention interpretation to find tau: the threshold that distinguishes potential causes from non-causal time series
    
    #if len(s)<=5:
    #    potentials = []
    #    for i in indices:
    #        if scores[i]>1.:
    #            potentials.append(i)
    #else:
    #    potentials = []
    #    gaps = []
    #    for i in range(len(s)-1):
    #        if s[i]<1.: #tau should be greater or equal to 1, so only consider scores >= 1
    #            break
    #        gap = s[i]-s[i+1]
    #        gaps.append(gap)
    #    sortgaps = sorted(gaps, reverse=True)
    #    
    #    for i in range(0, len(gaps)):
    #        largestgap = sortgaps[i]
    #        index = gaps.index(largestgap)
    #        ind = -1
    #        if index<((len(s)-1)/2): #gap should be in first half
    #            if index>2:
    #                ind=index #gap should have index > 0, except if second score <1
    #                break
    #    if ind<0:
    #        ind = 0
    #            
    #    potentials = indices[:ind+1].tolist()
    print("Potential causes: ", potentials)
    validated = copy.deepcopy(potentials)
    
    #Apply PIVM (permutes the values) to check if potential cause is true cause
    for idx in potentials:
        random.seed(seed)
        X_test2 = X_train.clone().cpu().numpy()
        random.shuffle(X_test2[:,idx,:][0])
        shuffled = torch.from_numpy(X_test2)
        if cuda:
            shuffled=shuffled.cuda()
        model.eval()
        output = model(shuffled)
        testloss = F.mse_loss(output, Y_train)
        testloss = testloss.cpu().data.item()
        
        diff = firstloss-realloss
        testdiff = firstloss-testloss
        
        # debugging
        print("Potential cause: ", idx, " - Test loss: ", testloss, " - Diff: ", testdiff, " - Significance: ", significance)
        print("First loss: ", firstloss, " - Real loss: ", realloss, " - Diff: ", diff)

        if testdiff>(diff*significance): 
            validated.remove(idx) 
    
 
    weights = []
    
    #Discover time delay between cause and effect by interpreting kernel weights
    for layer in range(layers):
        weight = model.dwn.network[layer].net[0].weight.abs().view(model.dwn.network[layer].net[0].weight.size()[0], model.dwn.network[layer].net[0].weight.size()[2])
        weights.append(weight)

    causeswithdelay = dict()    
    for v in validated: 
        totaldelay=0    
        for k in range(len(weights)):
            w=weights[k]
            row = w[v]
            twolargest = heapq.nlargest(2, row)
            m = twolargest[0]
            m2 = twolargest[1]
            if m > m2:
                index_max = len(row) - 1 - max(range(len(row)), key=row.__getitem__)
            else:
                #take first filter
                index_max=0
            delay = index_max *(dilation_c**k)
            totaldelay+=delay
        if targetidx != v:
            causeswithdelay[(targetidx, v)]=totaldelay
        else:
            causeswithdelay[(targetidx, v)]=totaldelay+1
    print("Validated causes: ", validated)
    
    return validated, causeswithdelay, realloss, scores

### Run TCDF

In [5]:
import argparse
import torch
import pandas as pd
import numpy as np
import networkx as nx
import pylab
import copy
import matplotlib.pyplot as plt
import os
import sys

# os.chdir(os.path.dirname(sys.argv[0])) #uncomment this line to run in VSCode

def check_positive(value):
    """Checks if argument is positive integer (larger than zero)."""
    ivalue = int(value)
    if ivalue <= 0:
         raise argparse.ArgumentTypeError("%s should be positive" % value)
    return ivalue

def check_zero_or_positive(value):
    """Checks if argument is positive integer (larger than or equal to zero)."""
    ivalue = int(value)
    if ivalue < 0:
         raise argparse.ArgumentTypeError("%s should be positive" % value)
    return ivalue

class StoreDictKeyPair(argparse.Action):
    """Creates dictionary containing datasets as keys and ground truth files as values."""
    def __call__(self, parser, namespace, values, option_string=None):
        my_dict = {}
        for kv in values.split(","):
            k,v = kv.split("=")
            my_dict[k] = v
        setattr(namespace, self.dest, my_dict)

def getextendeddelays(gtfile, columns):
    """Collects the total delay of indirect causal relationships."""
    gtdata = pd.read_csv(gtfile, header=None)

    readgt=dict()
    effects = gtdata[1]
    causes = gtdata[0]
    delays = gtdata[2]
    gtnrrelations = 0
    pairdelays = dict()
    for k in range(len(columns)):
        readgt[k]=[]
    for i in range(len(effects)):
        key=effects[i]
        value=causes[i]
        readgt[key].append(value)
        pairdelays[(key, value)]=delays[i]
        gtnrrelations+=1
    
    g = nx.DiGraph()
    g.add_nodes_from(readgt.keys())
    for e in readgt:
        cs = readgt[e]
        for c in cs:
            g.add_edge(c, e)

    extendedreadgt = copy.deepcopy(readgt)
    
    for c1 in range(len(columns)):
        for c2 in range(len(columns)):
            paths = list(nx.all_simple_paths(g, c1, c2, cutoff=2)) #indirect path max length 3, no cycles
            
            if len(paths)>0:
                for path in paths:
                    for p in path[:-1]:
                        if p not in extendedreadgt[path[-1]]:
                            extendedreadgt[path[-1]].append(p)
                            
    extendedgtdelays = dict()
    for effect in extendedreadgt:
        causes = extendedreadgt[effect]
        for cause in causes:
            if (effect, cause) in pairdelays:
                delay = pairdelays[(effect, cause)]
                extendedgtdelays[(effect, cause)]=[delay]
            else:
                #find extended delay
                paths = list(nx.all_simple_paths(g, cause, effect, cutoff=2)) #indirect path max length 3, no cycles
                extendedgtdelays[(effect, cause)]=[]
                for p in paths:
                    delay=0
                    for i in range(len(p)-1):
                        delay+=pairdelays[(p[i+1], p[i])]
                    extendedgtdelays[(effect, cause)].append(delay)

    return extendedgtdelays, readgt, extendedreadgt

def evaluate(gtfile, validatedcauses, columns):
    """Evaluates the results of TCDF by comparing it to the ground truth graph, and calculating precision, recall and F1-score. F1'-score, precision' and recall' include indirect causal relationships."""
    extendedgtdelays, readgt, extendedreadgt = getextendeddelays(gtfile, columns)
    FP=0
    FPdirect=0
    TPdirect=0
    TP=0
    FN=0
    FPs = []
    FPsdirect = []
    TPsdirect = []
    TPs = []
    FNs = []
    for key in readgt:
        for v in validatedcauses[key]:
            if v not in extendedreadgt[key]:
                FP+=1
                FPs.append((key,v))
            else:
                TP+=1
                TPs.append((key,v))
            if v not in readgt[key]:
                FPdirect+=1
                FPsdirect.append((key,v))
            else:
                TPdirect+=1
                TPsdirect.append((key,v))
        for v in readgt[key]:
            if v not in validatedcauses[key]:
                FN+=1
                FNs.append((key, v))
          
    print("Total False Positives': ", FP)
    print("Total True Positives': ", TP)
    print("Total False Negatives: ", FN)
    print("Total Direct False Positives: ", FPdirect)
    print("Total Direct True Positives: ", TPdirect)
    print("TPs': ", TPs)
    print("FPs': ", FPs)
    print("TPs direct: ", TPsdirect)
    print("FPs direct: ", FPsdirect)
    print("FNs: ", FNs)
    precision = recall = 0.

    if float(TP+FP)>0:
        precision = TP / float(TP+FP)
    print("Precision': ", precision)
    if float(TP + FN)>0:
        recall = TP / float(TP + FN)
    print("Recall': ", recall)
    if (precision + recall) > 0:
        F1 = 2 * (precision * recall) / (precision + recall)
    else:
        F1 = 0.
    print("F1' score: ", F1,"(includes direct and indirect causal relationships)")

    precision = recall = 0.
    if float(TPdirect+FPdirect)>0:
        precision = TPdirect / float(TPdirect+FPdirect)
    print("Precision: ", precision)
    if float(TPdirect + FN)>0:
        recall = TPdirect / float(TPdirect + FN)
    print("Recall: ", recall)
    if (precision + recall) > 0:
        F1direct = 2 * (precision * recall) / (precision + recall)
    else:
        F1direct = 0.
    print("F1 score: ", F1direct,"(includes only direct causal relationships)")
    return FP, TP, FPdirect, TPdirect, FN, FPs, FPsdirect, TPs, TPsdirect, FNs, F1, F1direct

def evaluatedelay(extendedgtdelays, alldelays, TPs, receptivefield):
    """Evaluates the delay discovery of TCDF by comparing the discovered time delays with the ground truth."""
    zeros = 0
    total = 0.
    for i in range(len(TPs)):
        tp=TPs[i]
        discovereddelay = alldelays[tp]
        gtdelays = extendedgtdelays[tp]
        for d in gtdelays:
            if d <= receptivefield:
                total+=1.
                error = d - discovereddelay
                if error == 0:
                    zeros+=1
                
            else:
                next
           
    if zeros==0:
        return 0.
    else:
        return zeros/float(total)


def runTCDF(datafile):
    """Loops through all variables in a dataset and return the discovered causes, time delays, losses, attention scores and variable names."""
    df_data = pd.read_csv(datafile)

    allcauses = dict()
    alldelays = dict()
    allreallosses=dict()
    allscores=dict()

    columns = list(df_data)
    for c in columns:
        idx = df_data.columns.get_loc(c)
        causes, causeswithdelay, realloss, scores = findcauses(c, cuda=cuda, epochs=nrepochs, 
        kernel_size=kernel_size, layers=levels, log_interval=loginterval, 
        lr=learningrate, optimizername=optimizername,
        seed=seed, dilation_c=dilation_c, significance=significance, file=datafile)

        allscores[idx]=scores
        allcauses[idx]=causes
        alldelays.update(causeswithdelay)
        allreallosses[idx]=realloss

    return allcauses, alldelays, allreallosses, allscores, columns

def plotgraph(stringdatafile,alldelays,columns):
    """Plots a temporal causal graph showing all discovered causal relationships annotated with the time delay between cause and effect."""
    G = nx.DiGraph()
    for c in columns:
        G.add_node(c)
    for pair in alldelays:
        p1,p2 = pair
        nodepair = (columns[p2], columns[p1])

        G.add_edges_from([nodepair],weight=alldelays[pair])
    
    edge_labels=dict([((u,v,),d['weight'])
                    for u,v,d in G.edges(data=True)])
    
    pos=nx.circular_layout(G)
    nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels)
    nx.draw(G,pos, node_color = 'white', edge_color='black',node_size=1000,with_labels = True)
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#200808") 

    pylab.show()







In [6]:

def main(datafiles, evaluation, levels, kernel_size, dilation_c, plot):
    if evaluation:
        totalF1direct = [] #contains F1-scores of all datasets
        totalF1 = [] #contains F1'-scores of all datasets

        receptivefield=1
        for l in range(0, levels):
            receptivefield+=(kernel_size-1) * dilation_c**(l)

    for datafile in datafiles.keys(): 
        stringdatafile = str(datafile)
        if '/' in stringdatafile:
            stringdatafile = str(datafile).rsplit('/', 1)[1]
        
        print("\n Dataset: ", stringdatafile)

        # run TCDF
        allcauses, alldelays, allreallosses, allscores, columns = runTCDF(datafile) #results of TCDF containing indices of causes and effects

        print("\n===================Results for", stringdatafile,"==================================")
        for pair in alldelays:
            print(columns[pair[1]], "causes", columns[pair[0]],"with a delay of",alldelays[pair],"time steps.")

        

        if evaluation:
            # evaluate TCDF by comparing discovered causes with ground truth
            print("\n===================Evaluation for", stringdatafile,"===============================")
            FP, TP, FPdirect, TPdirect, FN, FPs, FPsdirect, TPs, TPsdirect, FNs, F1, F1direct = evaluate(datafiles[datafile], allcauses, columns)
            totalF1.append(F1)
            totalF1direct.append(F1direct)

            # evaluate delay discovery
            extendeddelays, readgt, extendedreadgt = getextendeddelays(datafiles[datafile], columns)
            percentagecorrect = evaluatedelay(extendeddelays, alldelays, TPs, receptivefield)*100
            print("Percentage of delays that are correctly discovered: ", percentagecorrect,"%")
            
        print("==================================================================================")
        
        if plot:
            plotgraph(stringdatafile, alldelays, columns)

    # In case of multiple datasets, calculate average F1-score over all datasets and standard deviation
    if len(datafiles.keys())>1 and evaluation:  
        print("\nOverall Evaluation: \n")      
        print("F1' scores: ")
        for f in totalF1:
            print(f)
        print("Average F1': ", np.mean(totalF1))
        print("Standard Deviation F1': ", np.std(totalF1),"\n")
        print("F1 scores: ")
        for f in totalF1direct:
            print(f)
        print("Average F1: ", np.mean(totalF1direct))
        print("Standard Deviation F1: ", np.std(totalF1direct))

In [ ]:
kernel_size = 250
levels = 1
nrepochs = 50000 
learningrate = 0.001
optimizername = 'Adam'
dilation_c = kernel_size
loginterval = 500
seed= 1111
cuda= True
significance= .8
plot = True
ground_truth = None
data = ['test_norm/motor_simulation_Trj_4_e3_norm.csv']

args_dict = {
    'kernel_size': kernel_size,
    'levels': levels,
    'nrepochs': nrepochs,
    'learningrate': learningrate,
    'optimizername': optimizername,
    'dilation_c': dilation_c,
    'loginterval': loginterval,
    'seed': seed,
    'cuda': cuda,
    'significance': significance,
    'plot': plot
}

if ground_truth is not None:
    datafiles = ground_truth
    main(datafiles, evaluation=True)

else:
    datafiles = dict()
    for dataset in data:
        print("Dataset: ", dataset)
        datafiles[dataset]=""
    main(datafiles, evaluation=False, 
         levels=levels, kernel_size=kernel_size, dilation_c=dilation_c, plot=plot)

Dataset:  test_norm/motor_simulation_Trj_4_e3_norm.csv

 Dataset:  motor_simulation_Trj_4_e3_norm.csv

 Analysis started for target:  Id_current
Epoch:  1 [0%] 	Loss: 1.2569e-01 - 	Lambda: 9.9990e-07
Get sparsity stats:  {'mean_attention': 0.731019139289856, 'near_zero': 0, 'active': 10, 'active_idx': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'sparsity_ratio': 0.0}
Epoch: 500 [1%] 	Loss: 8.8001e-04 - 	Lambda: 9.5123e-07
Get sparsity stats:  {'mean_attention': 0.7361042499542236, 'near_zero': 0, 'active': 10, 'active_idx': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'sparsity_ratio': 0.0}
